# MXNET - MLP MODEL

In [1]:
import pandas as pd
import mxnet as mx
import numpy as np
import logging

pd.set_option('display.max_columns', 500)

from sklearn import preprocessing
from sklearn.utils import shuffle

## Parameters

In [2]:
LIMIT = 200
BATCH_SIZE = 500
BATCH_SIZE_TEST = 200
NUM_CLASSES = 2
LEARNING_RATE = 0.01
NUM_EPOCHS = 1000
DROPOUT=0.2

## Load Data

In [3]:
df = pd.read_csv('./data/data_processed_no_normalized.csv')

In [4]:
df = shuffle(df, random_state=0)

### Feature Engineering

In [5]:
def calculate_exp(x):
    return np.exp(x)

In [6]:
df['HP_1over2'] = df['HP_p1'] - df['HP_p2']
df['Attack_1over2'] = df['Attack_p1'] - df['Attack_p2']
df['Defense_1over2'] = df['Defense_p1'] - df['Defense_p2']
df['SpAtk_1over2'] = df['SpAtk_p1'] - df['SpAtk_p2']
df['SpDef_1over2'] = df['SpDef_p1'] - df['SpDef_p2']
df['Speed_1over2'] = df['Speed_p1'] - df['Speed_p2']

In [7]:
df['HP_2over1'] = df['HP_p2'] - df['HP_p1']
df['Attack_2over1'] = df['Attack_p2'] - df['Attack_p1']
df['Defense_2over1'] = df['Defense_p2'] - df['Defense_p1']
df['SpAtk_2over1'] = df['SpAtk_p2'] - df['SpAtk_p1']
df['SpDef_2over1'] = df['SpDef_p2'] - df['SpDef_p1']
df['Speed_2over1'] = df['Speed_p2'] - df['Speed_p1']

In [8]:
df['HP_1over2'] = df['HP_1over2'].apply(calculate_exp)
df['Attack_1over2'] = df['Attack_1over2'].apply(calculate_exp)
df['Defense_1over2'] = df['Defense_1over2'].apply(calculate_exp)
df['SpAtk_1over2'] = df['SpAtk_1over2'].apply(calculate_exp)
df['SpDef_1over2'] = df['SpDef_1over2'].apply(calculate_exp)
df['Speed_1over2'] = df['Speed_1over2'].apply(calculate_exp)

df['HP_2over1'] = df['HP_2over1'].apply(calculate_exp)
df['Attack_2over1'] = df['Attack_2over1'].apply(calculate_exp)
df['Defense_2over1'] = df['Defense_2over1'].apply(calculate_exp)
df['SpAtk_2over1'] = df['SpAtk_2over1'].apply(calculate_exp)
df['SpDef_2over1'] = df['SpDef_2over1'].apply(calculate_exp)
df['Speed_2over1'] = df['Speed_2over1'].apply(calculate_exp)

In [9]:
df_normalized = preprocessing.MinMaxScaler().fit_transform(df)
df = pd.DataFrame(df_normalized, columns=df.columns)

### Split Data & Label

In [10]:
df_label = df['label']
df_data = df.drop(['label'], axis=1)

### Split Train & Test

In [11]:
df_test_data = df_data[:LIMIT]
df_test_label = df_label[:LIMIT]
print "Test size: " + str(df_test_data.shape[0])

Test size: 200


In [12]:
df_train_data = df_data[LIMIT:]
df_train_label = df_label[LIMIT:]
print "Train size: " + str(df_train_data.shape[0])

Train size: 49800


### Initialize data iterators

In [13]:
train_iter = mx.io.NDArrayIter(np.array(df_train_data), np.array(df_train_label), BATCH_SIZE, shuffle=True)
val_iter = mx.io.NDArrayIter(np.array(df_test_data), np.array(df_test_label), BATCH_SIZE_TEST)

### Variables

In [14]:
data = mx.sym.var('data')
data = mx.sym.Dropout(data=data, p=DROPOUT)

### Network

In [ ]:
# The first fully-connected layer and the corresponding activation function
fc1  = mx.sym.FullyConnected(data=data, num_hidden=128)
act1 = mx.sym.Activation(data=fc1, act_type="relu")

# The second fully-connected layer and the corresponding activation function
fc2  = mx.sym.FullyConnected(data=act1, num_hidden = 64)
act2 = mx.sym.Activation(data=fc2, act_type="relu")

fc3  = mx.sym.FullyConnected(data=act2, num_hidden = 64)
act3 = mx.sym.Activation(data=fc3, act_type="relu")

fc4  = mx.sym.FullyConnected(data=act3, num_hidden = 64)
act4 = mx.sym.Activation(data=fc4, act_type="relu")

fc5  = mx.sym.FullyConnected(data=act4, num_hidden = 64)
act5 = mx.sym.Activation(data=fc5, act_type="relu")

# POKEMON has 2 classes
fc6  = mx.sym.FullyConnected(data=act5, num_hidden=NUM_CLASSES)
# Softmax with cross entropy loss
mlp  = mx.sym.SoftmaxOutput(data=fc6, name='softmax')

### Train

In [ ]:
#logging.getLogger().setLevel(logging.DEBUG)

mlp_model = mx.mod.Module(symbol=mlp, context=mx.cpu())
mlp_model.fit(train_iter,  # train data
              eval_data=val_iter,  # validation data
              optimizer='Adam',  # use Adam to train
              optimizer_params={'learning_rate': LEARNING_RATE},  # use fixed learning rate
              eval_metric='acc',  # report accuracy during training
              #batch_end_callback = mx.callback.Speedometer(BATCH_SIZE, 400), # output progress for each 100 data batches
              num_epoch=NUM_EPOCHS)  # train for at most 10 dataset passes

### Prediction

In [ ]:
test_iter = mx.io.NDArrayIter(np.array(df_test_data), np.array(df_test_label), BATCH_SIZE_TEST)
# predict accuracy of mlp
acc = mx.metric.Accuracy()
mlp_model.score(test_iter, acc)

In [ ]:
test_iter = mx.io.NDArrayIter(np.array(df_train_data), np.array(df_train_label), BATCH_SIZE)
# predict accuracy of mlp
acc = mx.metric.Accuracy()
mlp_model.score(test_iter, acc)